# Royal Family Calendar

This is an essential tool for all Royalists who wish to catch a glimpse of their favourite Royals!

Update the dates in the second cell and run the function in order to return a list of all upcoming events in your desired timeframe!
The function will even save a copy of the events in a csv file for you!

In [1]:
import requests
import re
from bs4 import BeautifulSoup
import pandas as pd
from geotext import GeoText

In [4]:
lst=[]
for i in range(12):
    lst.append('https://www.royal.uk/future-engagements?text=&mrf=&date%5Bmin%5D%5Bdate%5D=15/12/2019&date%5Bmax%5D%5Bdate%5D=15/01/2020&id=&page='+str(i))
lst

['https://www.royal.uk/future-engagements?text=&mrf=&date%5Bmin%5D%5Bdate%5D=15/12/2019&date%5Bmax%5D%5Bdate%5D=15/01/2020&id=&page=0',
 'https://www.royal.uk/future-engagements?text=&mrf=&date%5Bmin%5D%5Bdate%5D=15/12/2019&date%5Bmax%5D%5Bdate%5D=15/01/2020&id=&page=1',
 'https://www.royal.uk/future-engagements?text=&mrf=&date%5Bmin%5D%5Bdate%5D=15/12/2019&date%5Bmax%5D%5Bdate%5D=15/01/2020&id=&page=2',
 'https://www.royal.uk/future-engagements?text=&mrf=&date%5Bmin%5D%5Bdate%5D=15/12/2019&date%5Bmax%5D%5Bdate%5D=15/01/2020&id=&page=3',
 'https://www.royal.uk/future-engagements?text=&mrf=&date%5Bmin%5D%5Bdate%5D=15/12/2019&date%5Bmax%5D%5Bdate%5D=15/01/2020&id=&page=4',
 'https://www.royal.uk/future-engagements?text=&mrf=&date%5Bmin%5D%5Bdate%5D=15/12/2019&date%5Bmax%5D%5Bdate%5D=15/01/2020&id=&page=5',
 'https://www.royal.uk/future-engagements?text=&mrf=&date%5Bmin%5D%5Bdate%5D=15/12/2019&date%5Bmax%5D%5Bdate%5D=15/01/2020&id=&page=6',
 'https://www.royal.uk/future-engagements?text=&

In [15]:
def royal_events(lst):
    colnames = ["Date", "Member of Royal Family", "Event", "Location"]
    df = pd.DataFrame(columns=colnames)
    for url in lst:
        html = requests.get(url).content
        soup = BeautifulSoup(html)
        dates = soup.select("section ul li div span")
        dates = [i.string for i in dates]
        royals = soup.select("li h2")
        royals = [i.string for i in royals]
        events = soup.select("li div p")
        events = [i.string for i in events]

        new_events = []
        for event in events:
            m=re.search(r"(\bwill\b\s*(?:\S+\b\s*(.*?,|\W)){0,})",str(event)).group(0)
            new_events.append(m)
        events2 = [event.strip(",") for event in new_events]
        clean_events = [event.strip("will ") for event in events2]

        locations = []
        for event in events:
            if "," in event:
                event=event.split(", ")
                locations.append(event[-1])
            elif "at" in event:
                event=event.split(" at ")
                locations.append(event[-1])
            elif "in" in event:
                event=event.split(" in ")
                locations.append(event[-1])

        City = []
        for event in events:
            city = GeoText(event)
            City.append(city.cities)
        
        df1 = pd.DataFrame(columns=colnames)
        df1["Date"] = dates
        df1["Member of Royal Family"] = royals
        df1["Event"] = clean_events
        df1["Location"] = City
        df=df.append(df1)
    df = df.reset_index(drop=True)
    df.index = df.index + 1
    df.to_csv('Royal_Events.csv', sep=',')
    return df

royal_events(lst)

,Date,Member of Royal Family,Event,Location
1,15 December 2019,The Princess Royal,attend the BBC Sports Personality of the Year ...,"[Aberdeen, Aberdeen]"
2,17 December 2019,"The Duke of Gloucester, The Duchess of Gloucester",attend the London Fire Brigade's Carol Service...,[London]
3,7 January 2020,The Countess of Wessex,visit the teams and attend a training session ...,[Marlow]
4,8 January 2020,The Princess Royal,attend the Oxford Farming Conference Debate at...,"[Oxford, Oxford]"
5,8 January 2020,The Earl of Wessex,attend a Reception and Concert at Ronnie Scott...,[London]
6,10 January 2020,The Princess Royal,Chair a Council Meeting,[London]
7,14 January 2020,The Princess Royal,receive an Honorary Degree of Doctor of Laws a...,"[University, Aberdeen, Aberdeen]"
8,14 January 2020,The Duke of Gloucester,chair a panel of judges for the British Expert...,[London]
9,15 January 2020,The Princess Royal,visit the Metropolitan Police Service Special ...,[London]
10,15 January 2020,The Princess Royal,attend a Dinner at The Corporation of Trinity ...,[London]
